In [1]:
import numpy as np
import pandas as pd
from predictor.test_predictor import LinearRegressionPredictor
from predictor.train_models import train_linear_regression

In [2]:
model = LinearRegressionPredictor()
model.train_and_save_model() 

PANC
KBOI
KORD
KDEN
KDTW
PHNL
KIAH
KMIA
KMSP
KOKC
KBNA
KJFK
KPHX
KPWM
KPDX
KSLC
KSAN
KSFO
KSEA
KDCA


In [6]:
from predictor.utils import stations_list
for station in stations_list:
    print(model.predict(f"../data/restructured_simple/combined/{station}.csv",station))

       YEAR  MONTH  DAY  TMAX  TMIN  PRCP  SNOW  SNWD  TAVG
26475  2024     11   26   NaN   NaN   NaN   0.0   0.0   NaN
26476  2024     11   27   NaN   NaN   NaN   0.0   0.0   NaN
26477  2024     11   28   NaN   NaN   NaN   0.0   0.0   NaN
26478  2024     11   29   NaN   NaN   NaN   0.0   0.0   NaN
26479  2024     11   30   NaN   NaN   NaN   0.0   0.0   NaN


ValueError: Input X contains NaN.
LinearRegression does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [ ]:
data = pd.read_csv("regression_data/PANC.csv")

In [4]:
from analysis.evaluate_model import evaluate_model_years

In [5]:
evaluate_model_years(2018,2022, predictor=RandomForestPredictor())

array([47.63, 55.9 , 58.3 , 62.85, 52.7 ])